# 코드 설명 (임시)

- GAN Generator의 중간값 뽑아서 저장해놓는 코드

In [1]:
import tqdm

def nop(it, *a, **k):
    return it

real_tqdm = tqdm.tqdm
tqdm.tqdm = nop

import cv2
import torch
import time
import os
import glob



import numpy as np
np.bool = np.bool_

from utils.inference.image_processing import crop_face, get_final_image, show_images
from utils.inference.video_processing import read_video, get_target, get_final_video, add_audio_from_another_video, face_enhancement
from utils.inference.core import model_inference

from network.AEI_Net import AEI_Net
from coordinate_reg.image_infer import Handler
from insightface_func.face_detect_crop_multi import Face_detect_crop
from arcface_model.iresnet import iresnet100
from models.pix2pix_model import Pix2PixModel
from models.config_sr import TestOptions

#os.environ["CUDA_VISIBLE_DEVICES"] = "2"


/compuworks/anaconda3/envs/xaiseung_ghost_cuda114/lib/python3.9/site-packages/kornia/augmentation/augmentation.py:1830: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  warnings.warn(
/compuworks/anaconda3/envs/xaiseung_ghost_cuda114/lib/python3.9/site-packages/mxnet/numpy/fallback.py:143: DeprecationWarning: In accordance with NEP 32, the function mirr was removed from NumPy version 1.20.  A replacement for this function is available in the numpy_financial library: https://pypi.org/project/numpy-financial
  mirr = onp.mirr
/compuworks/anaconda3/envs/xaiseung_ghost_cuda114/lib/python3.9/site-packages/mxnet/numpy/fallback.py:160: DeprecationWarning: In accordance with NEP 32, the function npv was removed from NumPy version 1.20.  A replacement for this function is available in the numpy_financial library: https://pypi.org/project/numpy-financial
  npv = onp.npv
/compuworks/anaconda3/envs/xaise

### Load Models

In [2]:
app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

# main model for generation
G = AEI_Net(backbone='unet', num_blocks=2, c_id=512)
G.eval()
G.load_state_dict(torch.load('weights/G_unet_2blocks.pth', map_location=torch.device('cpu')))
G = G.cuda()
G = G.half()

# arcface model to get face embedding
netArc = iresnet100(fp16=False)
netArc.load_state_dict(torch.load('arcface_model/backbone.pth'))
netArc=netArc.cuda()
netArc.eval()

# model to get face landmarks
handler = Handler('./coordinate_reg/model/2d106det', 0, ctx_id=0, det_size=640)

# model to make superres of face, set use_sr=True if you want to use super resolution or use_sr=False if you don't
use_sr = True
if use_sr:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    torch.backends.cudnn.benchmark = True
    opt = TestOptions()
    #opt.which_epoch ='10_7'
    model = Pix2PixModel(opt)
    model.netG.train()

input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)


loading ./coordinate_reg/model/2d106det 0
input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)


[06:15:45] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.5.0. Attempting to upgrade...
[06:15:45] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[06:15:45] ../src/base.cc:79: cuDNN lib mismatch: linked-against version 8204 != compiled-against version 8101.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.


Network [LIPSPADEGenerator] was created. Total number of parameters: 72.2 million. To see the architecture, do print(network).
Load checkpoint from path:  weights/10_net_G.pth


# 중간계층값

In [7]:
import torch
import torch.nn.functional as F
import numpy as np
import pickle

from utils.inference.faceshifter_run import faceshifter_batch
from utils.inference.image_processing import crop_face, normalize_and_torch, normalize_and_torch_batch
from utils.inference.video_processing import read_video, crop_frames_and_get_transforms, resize_frames
from utils.inference.core import transform_target_to_torch


import time

set_target = False
half=True
similarity_th=0.15
crop_size = 224 # don't change this
BS = 60

save_dir = "./examples/z_embeds/VggFace2-crop"
os.makedirs(save_dir, exist_ok=True)


np.random.seed(7777)

id_folder_list = glob.glob("./examples/images/VggFace2-crop/*")
id_folder_list.sort()
with open(f"{save_dir}/VggFace2-cropzattr_embed_list.txt", "w") as embed_log:
    output_cnt = 0
    for id_folder in real_tqdm(id_folder_list):
        pics = glob.glob(id_folder+"/*.jpg")
        pics.sort()
        np.random.shuffle(pics)
        for pic in pics[:2]:
            source_full = cv2.imread(pic)
            full_frames = [source_full]
            #print(source_full.shape)
            #source = crop_face(source_full, app, crop_size)[0]
            # 버그 수정. target은 bgr 형태로 입력이 되어야 함
            source_curr = cv2.resize(source_full, (crop_size, crop_size))
            target = [source_curr.copy()]
            source_curr = normalize_and_torch(source_curr[:,:,::-1])
            #print(source_curr.shape)
            source_embed = netArc(F.interpolate(source_curr, scale_factor=0.5, mode='bilinear', align_corners=True))


            #get_target(full_frames, app, crop_size)
            target_norm = normalize_and_torch_batch(np.array(target))

            target_embeds = netArc(F.interpolate(target_norm, scale_factor=0.5, mode='bilinear', align_corners=True))

            # Get the cropped faces from original frames and transformations to get those crops
            #crop_frames_list, tfm_array_list = crop_frames_and_get_transforms(full_frames, target_embeds, app, netArc, crop_size, set_target, similarity_th=similarity_th)
            #crop_frames = crop_frames_list[0]

            crop_frames = target
            resized_frs, present = resize_frames(crop_frames)
            resized_frs = np.array(resized_frs)


            target_batch_rs = transform_target_to_torch(resized_frs, half=half)
            #assert False
            if half:
                source_embed = source_embed.half()
            
            bs = target_batch_rs.shape[0]
            source_emb = torch.cat([source_embed]*bs)
            
            #Y_st = faceshifter_batch(source_embed, target_batch_rs, G)
            Y_st, zattrs = G(target_batch_rs, source_emb)
            zattrs = list(zattrs)
            for i in range(len(zattrs)):
                zattrs[i] = zattrs[i].detach().cpu().numpy()
            #print(source_embed.shape)

            with open(f"{save_dir}/{output_cnt:d}.pkl", "wb") as file:
                pickle.dump(zattrs, file)
            embed_log.write(pic+"\n")
            #assert False
            output_cnt += 1
            #if output_cnt >= 10:
            #    assert False;

100%|██████████| 8631/8631 [25:23<00:00,  5.67it/s]
